Lien du site : 
https://www.lacentrale.fr/auto-occasion-annonce-69113195868.html

# Scrappe la centrale


### Import

In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import pandas as pd
import urllib
import re
import requests
import csv
import time

from selenium.webdriver.common.service import Service
from selenium.webdriver.common.service import Service
from fake_useragent import UserAgent
from urllib3.exceptions import NewConnectionError
import logging
from datetime import datetime, timedelta
from selenium.common.exceptions import ElementClickInterceptedException

import pandas as pd 
import numpy as np 

import random
from selenium.webdriver.common.action_chains import ActionChains
from itertools import combinations

### Scrappe

Fake Action

In [2]:
def action_aleatoire(driver):
    actions_possibles = ["scroll", "click_button", "fill_text"]
    action_choisie = random.choice(actions_possibles)

    # Enregistrer la position actuelle de la page
    current_scroll_position = driver.execute_script("return window.pageYOffset;")

    if action_choisie == "scroll":
        # Action aléatoire : Faites défiler la page vers le bas
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    elif action_choisie == "click_button":
        # Action aléatoire : Cliquez sur un élément aléatoire (par exemple, un bouton de tri)
        wait = WebDriverWait(driver, 10)
        elements = wait.until(EC.presence_of_all_elements_located((By.XPATH, ".//button[@class='Iqt3 Iqt3-mod-bold Button-No-Standard-Style Iqt3-mod-variant-outline Iqt3-mod-theme-none Iqt3-mod-shape-rounded-small Iqt3-mod-shape-mod-default Iqt3-mod-spacing-default Iqt3-mod-size-default']")))
        
        if elements:
            random_button = random.choice(elements)
            try:
                random_button.click()
            except ElementClickInterceptedException:
                pass  # Gérer l'exception, par exemple, en ne faisant rien

    elif action_choisie == "fill_text":
        # Action aléatoire : Remplissez un champ de texte (par exemple, une recherche)
        wait = WebDriverWait(driver, 10)
        try:
            search_input = wait.until(EC.visibility_of_element_located((By.XPATH, ".//input[@class='k_my-input']")))
            search_terms = ["Amsterdam", "Brussels", "Dublin", "Berlin", "Copenhagen", "Oslo", "Stockholm", "Helsinki", "Reykjavik", "Moscow", "Warsaw", "Paris", "Rome", "Madrid", "London", "Lisbon", "Prague", "Budapest", "Vienna", "Athens", "Bucharest", "Sofia", "Belgrade", "Zagreb", "Tirana", "Skopje", "Podgorica", "Pristina", "Tallinn", "Riga", "Vilnius", "Chișinău", "Minsk", "Kyiv", "Tbilisi", "Yerevan", "Baku", "Ankara", "Ercan", "Valletta", "Vatican City", "Monaco", "Rimini"]
            random_search_term = random.choice(search_terms)
            search_input.send_keys(random_search_term)
        except Exception as e:
            pass  # Gérer l'exception, par exemple, en ne faisant rien

    # Revenir à la position enregistrée
    driver.execute_script(f"window.scrollTo(0, {current_scroll_position});")

Scrape

In [16]:
import time
import random
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, StaleElementReferenceException
from fake_useragent import UserAgent

# Configurations du navigateur
ua = UserAgent()
user_agent = ua.random
options = webdriver.FirefoxOptions()
options.add_argument('--headless')
options.add_argument(f'user-agent={user_agent}')

# Initialisation du navigateur
driver = webdriver.Firefox(options=options)
base_url = "https://www.lacentrale.fr/listing?makesModelsCommercialNames=&options=&page="
xpath_expression = "//a[contains(@class, 'Vehiculecard_Vehiculecard_vehiculeCard Containers_Containers_containers Containers_Containers_borderRadius Containers_Containers_darkShadowWide')]"

# Numéro de la première page à visiter
current_page = 0

# Boucle pour parcourir les pages
while True:
    # Construire l'URL de la page actuelle
    all_links = []
    url = base_url + str(current_page)
    time.sleep(random.uniform(2, 5))

    # Charger la page web
    driver.get(url)

    try:
        # Insérer ici le code pour extraire les liens ou effectuer d'autres opérations sur la page
        links = driver.find_elements(By.XPATH, xpath_expression)

        # Ajouter les liens à la liste
        all_links.extend(link.get_attribute("href") for link in links)

        # Parcourir chaque lien pour effectuer des opérations
        for link in all_links:
            try:
                # Afficher le lien trouvé
                print("Lien trouvé :", link)

                # Accéder à la page du lien
                driver.get(link)
                time.sleep(random.uniform(2, 5))

                # Insérer ici le code pour effectuer des opérations sur la page du lien

                try:
                    nom_xpath = "//div[contains(@class, 'Text_Text_text SummaryInformation_title__5CYhW Text_Text_headline3')]"
                    
                    nom = driver.find_element(By.XPATH, nom_xpath).text
                    print("Nom du véhicule:", nom)
                    time.sleep(random.uniform(2, 5))
                except NoSuchElementException:
                    # Si la div n'est pas trouvée sur la page du lien, afficher un message
                    nom = " "
                    print("aucun nom trouvé")
                try:
                    carac_xpath = "//div[contains(@class, 'Text_Text_text SummaryInformation_subtitle__M7MAb Text_Text_body2')]"
                    carac = driver.find_element(By.XPATH, carac_xpath).text
                    print("Carac:", carac)

                    # Diviser le texte en éléments
                    elements = carac.split()

                    # Variables pour stocker les informations
                    cylindree_moteur = ""
                    type_moteur = ""
                    puissance = ""
                    finition = ""

                    # Utiliser un indicateur pour savoir quel type d'information on traite actuellement
                    current_type = None

                    # Parcourir les éléments et attribuer aux variables appropriées
                    for element in elements:
                        # Vérifier s'il s'agit de la cylindrée du moteur (exprimée en centimètres cubes)
                        if element.replace(".", "").isdigit() and not cylindree_moteur:
                            cylindree_moteur = element
                        # Vérifier s'il s'agit du type de moteur (contient des lettres)
                        elif any(char.isalpha() for char in element) and not type_moteur:
                            type_moteur = element
                        # Vérifier s'il s'agit de la puissance (contient des chiffres)
                        elif element.isdigit() and not puissance:
                            puissance = element
                        # Si ce n'est ni la cylindrée, ni le type, ni la puissance, c'est probablement la finition
                        elif not finition:
                            finition += element + " "

                    # Afficher les résultats
                    print("Cylindrée du moteur:", cylindree_moteur.strip())
                    print("Type de moteur:", type_moteur.strip())
                    print("Puissance:", puissance.strip())
                    print("Finition:", finition.strip())

                except NoSuchElementException:
                    # Si la div n'est pas trouvée sur la page du lien, afficher un message
                    carac = " "
                    print("Carac non trouvé")
                try:
                    prix_xpath = "//span[contains(@class, 'PriceInformation_classifiedPrice__b-Jae')]"
                    
                    prix = driver.find_element(By.XPATH, prix_xpath).text
                    print("prix:", prix)
                    time.sleep(random.uniform(2, 5))
                except NoSuchElementException:
                    # Si la div n'est pas trouvée sur la page du lien, afficher un message
                    prix = " "
                    print("aucun nom trouvé")
                try:
                    duree_publi_xpath = "//div[contains(@class, 'Text_Text_text Text_Text_body2')]"
                    
                    duree_publi = driver.find_element(By.XPATH, duree_publi_xpath).text
                    print("publié depuis:",  duree_publi)
                    time.sleep(random.uniform(2, 5))
                except NoSuchElementException:
                    # Si la div n'est pas trouvée sur la page du lien, afficher un message
                    durée_publi = " "
                    print("no durée publi renseigné")
                try:
                    spans_xpath = "//span[contains(@class, 'Text_Text_text Icon-button_IconButton_label Text_Text_label2')]"

                    spans = driver.find_elements(By.XPATH, spans_xpath)
                    print(spans)
    # Nombre de spans
                    num_spans = len(spans)

    # Créer des variables dynamiquement
                    for index, span in enumerate(spans, start=1):
        # Utilisation de JavaScript pour extraire le texte
                        span_text = driver.execute_script("return arguments[0].textContent;", span)
                        variable_name = f"point_fort{index}"
                        locals()[variable_name] = span_text.strip()
                        print(f"{variable_name}:", locals()[variable_name])

                        time.sleep(random.uniform(2, 5))

                except NoSuchElementException:
    # Si la div n'est pas trouvée sur la page du lien, afficher un message
                    for index in range(1, num_spans + 1):
                        variable_name = f"point_fort{index}"
                        locals()[variable_name] = " "
                        print(f"{variable_name}: not found")
                try:
                 ########################EN COURS######################   
                    h3_xpath = "//h3[contains(@class, 'Text_Text_text Text_Text_subtitle1')]"
                    h3_element = driver.find_element(By.XPATH, h3_xpath)

                    
                    ul_element = h3_element.find_element(By.XPATH, "following-sibling::ul")

                   
                    li_elements = ul_element.find_elements(By.XPATH, "li")

                   
                    for index, li_element in enumerate(li_elements, start=1):
                        li_text = li_element.text
                        print(f"Texte du li {index}: {li_text}")

                    time.sleep(random.uniform(2, 5))

                except NoSuchElementException:
                    print("H3, UL ou LI non trouvé")





            except StaleElementReferenceException:
                # Si l'élément de lien est obsolète, afficher un message
                print("StaleElementReferenceException pour le lien :", link)

    except NoSuchElementException:
        # Si l'élément n'est pas trouvé, cela signifie probablement que la page n'existe pas
        print("Fin de la boucle : La page", current_page, "n'existe pas.")
        break

    # Passer à la page suivante
    current_page += 1

# Fermer le navigateur à la fin
driver.quit()


Lien trouvé : https://www.lacentrale.fr/auto-occasion-annonce-69113063641.html
Nom du véhicule: CITROEN C5 AIRCROSS phase 2
Carac: 1.5 BLUEHDI 131 SHINE PACK
Cylindrée du moteur: 1.5
Type de moteur: BLUEHDI
Puissance: 131
Finition: SHINE
prix: 36 360 €
publié depuis: Publiée il y a 28 jours
[<selenium.webdriver.remote.webelement.WebElement (session="525783fc-11fe-4ca4-ae10-9685b24c2d7f", element="090bc2d8-b65a-48c2-977e-392d289a91b4")>, <selenium.webdriver.remote.webelement.WebElement (session="525783fc-11fe-4ca4-ae10-9685b24c2d7f", element="f097b6bd-039d-4e14-84c3-31feac9422f4")>, <selenium.webdriver.remote.webelement.WebElement (session="525783fc-11fe-4ca4-ae10-9685b24c2d7f", element="902e6988-689b-4346-a142-bf752a43f12f")>, <selenium.webdriver.remote.webelement.WebElement (session="525783fc-11fe-4ca4-ae10-9685b24c2d7f", element="312e4e5b-5f81-44c7-9686-8598494000b8")>, <selenium.webdriver.remote.webelement.WebElement (session="525783fc-11fe-4ca4-ae10-9685b24c2d7f", element="628963c7

KeyboardInterrupt: 